In [2]:
#use rfm model to Analyze user value
import time 
import numpy as np  
import pandas as pd 
import mysql.connector  

In [3]:
data = pd.read_csv('../../data/sales.csv',index_col='USERID')

In [6]:
data.dtypes

ORDERDATE      object
ORDERID         int64
AMOUNTINFO    float64
dtype: object

In [8]:
data.head()

,ORDERDATE,ORDERID,AMOUNTINFO
USERID,,,
142074,2016-01-01,4196439032,9399.0
56927,2016-01-01,4198324983,8799.0
87058,2016-01-01,4191287379,6899.0
136104,2016-01-01,4198508313,5999.0
117831,2016-01-01,4202238313,5399.0


In [9]:
data.ORDERID=data.ORDERID.astype("object")

In [12]:
data.describe()


,AMOUNTINFO
count,86127.000000
mean,744.705249
std,1425.211176
min,0.500000
25%,13.000000
50%,59.000000
75%,629.000000
max,30999.000000


In [13]:
data.isnull().sum()

ORDERDATE     2
ORDERID       0
AMOUNTINFO    8
dtype: int64

In [15]:
#drop null value and <1 values
data = data.dropna() 
data = data[data['AMOUNTINFO'] > 1] 


In [16]:
data['ORDERDATE'] = pd.to_datetime(data['ORDERDATE'], format='%Y-%m-%d')

In [18]:
data.head()

,ORDERDATE,ORDERID,AMOUNTINFO
USERID,,,
142074,2016-01-01,4196439032,9399.0
56927,2016-01-01,4198324983,8799.0
87058,2016-01-01,4191287379,6899.0
136104,2016-01-01,4198508313,5999.0
117831,2016-01-01,4202238313,5399.0


In [23]:
# data transformation
recency_value = data['ORDERDATE'].groupby(data.index).max()  
frequency_value = data['ORDERDATE'].groupby(data.index).count()  
monetary_value = data['AMOUNTINFO'].groupby(data.index).sum()  

In [24]:
recency_value

USERID
51220    2016-09-20
51221    2016-05-16
51224    2016-07-24
51225    2016-09-07
51226    2016-02-01
51227    2016-04-22
51229    2016-06-19
51230    2016-02-18
51231    2016-07-02
51232    2016-12-09
51235    2016-01-17
51243    2016-09-11
51244    2016-12-23
51245    2016-08-10
51246    2016-09-17
51248    2016-08-04
51250    2016-08-09
51251    2016-12-11
51255    2016-12-25
51258    2016-03-06
51259    2016-11-13
51260    2016-07-21
51263    2016-09-25
51264    2016-09-13
51267    2016-12-20
51273    2016-09-16
51274    2016-12-04
51275    2016-04-14
51277    2016-01-17
51278    2016-08-07
            ...    
165051   2016-11-21
165052   2016-11-15
165055   2016-08-14
165059   2016-09-17
165060   2016-06-01
165061   2016-12-19
165062   2016-08-28
165063   2016-08-19
165065   2016-12-28
165066   2016-05-14
165071   2016-05-18
165072   2016-08-28
165073   2016-12-12
165075   2016-12-17
165076   2016-11-03
165078   2016-06-11
165081   2016-02-11
165082   2016-09-15
165083   2016

In [27]:
# count r,f,m values
deadline_date = pd.datetime(2017,1,1)
r_interval = (deadline_date - recency_value).dt.days  
r_score = pd.cut(r_interval, 5, labels=[5, 4, 3, 2, 1]) 
f_score = pd.cut(frequency_value, 5, labels=[1, 2, 3, 4, 5]) 
m_score = pd.cut(monetary_value, 5, labels=[1, 2, 3, 4, 5]) 

In [29]:
# merge R、F、M 
rfm_list = [r_score, f_score, m_score] 
rfm_cols = ['r_score', 'f_score', 'm_score'] 
rfm_pd = pd.DataFrame(np.array(rfm_list).transpose(), dtype=np.int32, columns=rfm_cols,
                      index=frequency_value.index)  


In [30]:
rfm_pd

,r_score,f_score,m_score
USERID,,,
51220,4,1,1
51221,2,1,1
51224,3,1,1
51225,4,1,1
51226,1,1,1
51227,2,1,1
51229,3,1,1
51230,1,1,1
51231,3,1,1
